In [1]:
import pandas as pd
from datasets import load_from_disk

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
split_ds_path='/Users/markjos/projects/malachor5/data/pyarrow-datasets/tira-clean-split'

In [3]:
ds=load_from_disk(split_ds_path)
ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'start', 'end', 'transcription', 'eaf_source', 'wav_source', 'raw_transcription', 'clip', 'wav_rawpath', 'path', 'allosaurus', 'clap_ipa_cos_sim', 'wada_snr', 'nist_stnr', 'speaker-diarization-3.1', 'voice-activity-detection', 'whisper-large-v3', 'clapipa-transcription-allosaurus', 'vad_s', 'drz_s', 'trans_len', 'pcnt_speech', 'trans_len_sq', 'trans_len_log', 'cos_sim_softmax', 'cos_sim_log', 'duration'],
        num_rows: 16384
    })
    validation: Dataset({
        features: ['audio', 'start', 'end', 'transcription', 'eaf_source', 'wav_source', 'raw_transcription', 'clip', 'wav_rawpath', 'path', 'allosaurus', 'clap_ipa_cos_sim', 'wada_snr', 'nist_stnr', 'speaker-diarization-3.1', 'voice-activity-detection', 'whisper-large-v3', 'clapipa-transcription-allosaurus', 'vad_s', 'drz_s', 'trans_len', 'pcnt_speech', 'trans_len_sq', 'trans_len_log', 'cos_sim_softmax', 'cos_sim_log', 'duration'],
        num_rows: 2048
    })
    

In [6]:
# df=pd.DataFrame(columns=ds['train'].column_names+['split'])
split_dfs=[]
for split in ds:
    split_df=ds[split].remove_columns('audio').to_pandas()
    split_df['split']=split
    split_dfs.append(split_df)
df=pd.concat(split_dfs)
print(df.shape)
df.head()

(20480, 27)


,start,end,transcription,eaf_source,wav_source,raw_transcription,clip,wav_rawpath,path,allosaurus,...,vad_s,drz_s,trans_len,pcnt_speech,trans_len_sq,trans_len_log,cos_sim_softmax,cos_sim_log,duration,split
0,217011.0,219552.0,àprí jɜ̀dí ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jɜ̀dí ðáŋàlà,data/tira-asr/clips/HH01082021-m03s37ms011-m03...,/Users/markjos/Library/CloudStorage/GoogleDriv...,HH01082021-m03s37ms011-m03s39ms552.wav,a e i a d i ɡ a m a l a,...,2.379,2.447,23,0.936246,3.135494,3.135494,0.000044,-10.039731,2.541,train
1,221371.0,225023.0,àprí jɜ̀dí ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jɜ̀dí ðáŋàlà,data/tira-asr/clips/HH01082021-m03s41ms371-m03...,/Users/markjos/Library/CloudStorage/GoogleDriv...,HH01082021-m03s41ms371-m03s45ms023.wav,a j a d i v a n a l a,...,3.375,3.510,23,0.924151,3.135494,3.135494,0.000044,-10.028157,3.652,train
2,283401.0,286528.0,àprí jə̀və̀lɛ̀ðɔ́ ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jə̀və̀lɛ̀ðɔ́ ðáŋàlà,data/tira-asr/clips/HH01082021-m04s43ms401-m04...,/Users/markjos/Library/CloudStorage/GoogleDriv...,HH01082021-m04s43ms401-m04s46ms528.wav,a p e i e p e l e ð a l a m a l a,...,3.088,3.037,29,0.987528,3.367296,3.367296,0.000042,-10.080580,3.127,train
3,288835.0,291580.0,àprí jə̀və̀lɛ̀ðɔ́ ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jə̀və̀lɛ̀ðɔ́ ðáŋàlà,data/tira-asr/clips/HH01082021-m04s48ms835-m04...,/Users/markjos/Library/CloudStorage/GoogleDriv...,HH01082021-m04s48ms835-m04s51ms580.wav,a t e i j a p e d e ð a ɡ a n a ð a,...,2.413,2.649,29,0.879053,3.367296,3.367296,0.000044,-10.025673,2.745,train
4,304737.0,307762.0,àprí jə̀və̀lɛ̀ðɔ́ ðáŋàlà,/Users/markjos/Library/CloudStorage/GoogleDriv...,/Users/markjos/Library/CloudStorage/GoogleDriv...,àprí jə̀və̀lɛ̀ðɔ́ ðáŋàlà,data/tira-asr/clips/HH01082021-m05s04ms737-m05...,/Users/markjos/Library/CloudStorage/GoogleDriv...,HH01082021-m05s04ms737-m05s07ms762.wav,a e j e ð e d e ð a l a m a l a,...,2.920,2.919,29,0.965289,3.367296,3.367296,0.000042,-10.068530,3.025,train


# There are 45 ungrammatical sentences in the dataset

In [16]:
df['raw_transcription'].str.contains('\*').value_counts()

raw_transcription
False    20435
True        45
Name: count, dtype: int64

In [23]:
train=df['split']=='train'
validation=df['split']=='validation'
test=df['split']=='test'

train_wavs=pd.Series(df[train]['wav_source'].unique())
val_wavs=pd.Series(df[validation]['wav_source'].unique())
test_wavs=pd.Series(df[test]['wav_source'].unique())

df['split']
train_only = train_wavs[~train_wavs.isin(val_wavs) & ~train_wavs.isin(test_wavs)]
val_only = val_wavs[~val_wavs.isin(train_wavs) & ~val_wavs.isin(test_wavs)]
test_only = test_wavs[~test_wavs.isin(val_wavs) & ~test_wavs.isin(train_wavs)]

train_only.shape, val_only.shape, test_only.shape

((21,), (0,), (0,))